In [8]:
# Input and output vectors are given. 
inp = [ 0.7300, -1.0400, -1.2300,  1.6700, -0.6300,  1.4300, -0.8400,  0.1500,
         -2.3000,  3.1000, -1.4500, -1.8100,  1.8700, -0.1100, -0.2800,  1.1200,
         -0.4200,  2.8900]
out = [ 1.43,  10.1,  8.3,  1.03,  10.21, -0.1,  8.92,  5.1,
         -7.53, 34.72,  7.61,  3.2,  2.19,  7.15,  7.69, -0.18,
          8.81, 23.1]

In [9]:
# Define the polynomial model of degree 3, i.e., having 3 weights and 1 bias. 
# Also define the loss function
def polynomial_model(x, weight1, weight2, weight3, bias):
    return weight1 * x**3 + weight2 * x**2 + weight3 * x + bias

def loss(predictions, targets):
    return sum((predictions - targets)**2) / len(predictions)

In [5]:
# Define gradient manually wrt the exisiting parameters
# Note: You need to define appropriate derivative functions to define the gradient
# Use the defined gradient function to define the training function 
# Note: You cannot use autograd and optimizers
# Run it on the input and output vector with appropriate learning rate and number of iterations
# Plot the learned curve


In [ ]:
# Use PyTorch's autograd to automatically compute the gradients 
# Define the training function
# Note: You cannot use optimizers.
# Run it on the input and output vector with appropriate learning rate and number of iterations
# Plot the learned curve
import torch

inp = torch.tensor([ 0.7300, -1.0400, -1.2300,  1.6700, -0.6300,  1.4300, -0.8400,  0.1500,
                    -2.3000,  3.1000, -1.4500, -1.8100,  1.8700, -0.1100, -0.2800,  1.1200,
                    -0.4200,  2.8900], requires_grad=True)
out = torch.tensor([ 1.43,  10.1,  8.3,  1.03,  10.21, -0.1,  8.92,  5.1,
                    -7.53, 34.72,  7.61,  3.2,  2.19,  7.15,  7.69, -0.18,
                    8.81, 23.1], requires_grad=False)
def model(x, w, b):
    return w * x + b
    
def mse_loss(y_pred, y_true):
    return ((y_pred - y_true) ** 2).mean()
    
def train(x, y, epochs, lr):
    w = torch.randn(1, requires_grad=True)
    b = torch.randn(1, requires_grad=True)

    for epoch in range(epochs):
        y_pred = model(x, w, b)
        loss = mse_loss(y_pred, y)

        # Compute gradients
        loss.backward()

        # Manual gradient descent
        with torch.nograd():
            w -= lr * w.grad
            b -= lr * b.grad

            # Zero gradients
            w.grad.zero()
            b.grad.zero_()

    return w, b
learning_rate = 0.01
epochs = 1000

w, b = train(inp, out, epochs, learning_rate)

predictions = model(inp, w, b).detach().numpy()

plt.scatter(inp.numpy(), out.numpy(), color='blue', label='Actual data')
plt.plot(inp.numpy(), predictions, color='red', label='Fitted line')
plt.xlabel('Input')
plt.ylabel('Output')
plt.title('Linear Regression using Gradient Descent')
plt.legend()
plt.show()


In [ ]:
# Use PyTorch's autograd to automatically compute the gradients 
# Use optimizers to abstract how parameters get updated
# Define the training function
# Run it on the input and output vector with appropriate learning rate, number of iterations, and SGD optimizer
# Plot the learned curve

In [ ]:
# Split the dataset to training set (80%) and validation set (20%) randomly
# Use PyTorch's autograd to automatically compute the gradients 
# Use optimizers to abstract how parameters get updated
# Define the training function that tracks both training and validation losses
# Run it on the input and output vector with appropriate learning rate, number of iterations, and SGD optimizer
# Plot the learned curve
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

# Data
inp = torch.tensor([ 0.7300, -1.0400, -1.2300,  1.6700, -0.6300,  1.4300, -0.8400,  0.1500,
                    -2.3000,  3.1000, -1.4500, -1.8100,  1.8700, -0.1100, -0.2800,  1.1200,
                    -0.4200,  2.8900], dtype=torch.float32).view(-1, 1)
out = torch.tensor([ 1.43,  10.1,  8.3,  1.03,  10.21, -0.1,  8.92,  5.1,
                    -7.53, 34.72,  7.61,  3.2,  2.19,  7.15,  7.69, -0.18,
                    8.81, 23.1], dtype=torch.float32).view(-1, 1)

# Splitting data into training and validation sets
dataset = torch.utils.data.TensorDataset(inp, out)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=train_size)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=val_size)

# Model definition
class LinearRegression(nn.Module):
    def __init__(self):
        super(LinearRegression, self).__init__()
        self.linear = nn.Linear(1, 1)

    def forward(self, x):
        return self.linear(x)

# Training function
def train(model, criterion, optimizer, train_loader, val_loader, epochs):
    train_losses = []
    val_losses = []

    for epoch in range(epochs):
        model.train()

        # Training loop
        for x_train, y_train in train_loader:
            optimizer.zero_grad()
            y_pred_train = model(x_train)
            loss_train = criterion(y_pred_train, y_train)
            loss_train.backward()
            optimizer.step()

            train_losses.append(loss_train.item())

        model.eval()

        # Validation loop
        with torch.no_grad():
            for x_val, y_val in val_loader:
                y_pred_val = model(x_val)
                loss_val = criterion(y_pred_val, y_val)

                val_losses.append(loss_val.item())

    return train_losses, val_losses

# Model, Loss, Optimizer
model = LinearRegression()
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Train
epochs = 1000
train_losses, val_losses = train(model, criterion, optimizer, train_loader, val_loader, epochs)

# Predict
with torch.no_grad():
    predictions = model(inp)

# Plot learned curve
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.scatter(inp.numpy(), out.numpy(), color='blue', label='Actual data')
plt.plot(inp.numpy(), predictions.numpy(), color='red', label='Fitted line')
plt.xlabel('Input')
plt.ylabel('Output')
plt.title('Linear Regression using SGD Optimizer')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(train_losses, label='Training loss')
plt.plot(val_losses, label='Validation loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Losses')
plt.legend()

plt.tight_layout()
plt.show()